In [ ]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
import sys

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

import numpy as np
import random
import copy
import torch.optim as optim
import pickle
import os
import pandas as pd
import time
import matplotlib.pyplot as plt


In [ ]:
sys.path.append("/Volumes/GoogleDrive/My Drive/Colab Notebooks/NeuralNetwork")
from library_model import layers as lay
from library_model import model_building as mb
from library_model import model_training as mt
from data import data_loading as dt
from data import text as txt

In [ ]:
%%bash
pip install torchdata

In [ ]:
%%bash
pip install torchtext

In [ ]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')

In [ ]:
dataset_size = sum((1 for _ in train_iter))
dataset_size

In [ ]:
states = [lay.Network_state() for _ in range(5)]
for k, state in enumerate(states):
    p= state.parameters
    tr= state.training

    p.d_model=200+50*k
    p.d_hid=200+50*k
    p.nheads=2 + k//2
    p.d_key = p.d_model //p.nheads
    p.nlayers=2
    p.network = "encoder"


    tr.batch_size=20
    tr.data_split = 0.8 #fraction of dataset that will be used for training vs validation
    tr.lr = 1.
    tr.seq_length=35
    tr.optimizer = "sgd"

In [ ]:
datasets = [dt.Data() for _ in range(len(states))]
for d, s in zip(datasets, states):
    params = lay.param_count(s)
    s.training.data_fraction = lay.param_count(states[0])/params
    d.tokenizer, d.train_dataloader, d.test_dataloader = dt.process_data(train_iter, txt.library_text_coders, s, network= "encoder")
    s.parameters.ntokens = s.parameters.ntokens_out = len(d.tokenizer.vocab)
    s.training.schedule = mb.learning_rate_step(factor=5., drop=2., time=1.)

dataset_sizes = [len(d.train_dataloader) for d in datasets]
parameter_counts = list( map(lay.param_count, states))
compute = [6*a*b*0.8 for a, b in zip( dataset_sizes, parameter_counts)]
print(f"{compute[0]:.2E}")

In [ ]:
criterion = nn.CrossEntropyLoss()
models = []
trainers = []
for state in states:
    parts = mb.get_transformer_parts(state)
    model = lay.EncoderModel(parts.encoder, parts.linear).to(state.device)
    opt, scheduler = mb.get_optimizer(state, model)
    trainer = mt.Model_training(model, opt, criterion, scheduler)
    models.append(model)
    trainers.append(trainer)


In [ ]:
trainers[0].scheduler

In [ ]:
trainers[0](epochs = 5, data = datasets[0])